# Media Pipe + Data Modelling for Model Creation

In [165]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Dropout, Flatten, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.callbacks import EarlyStopping

## Read this:

### check all paths before running

## Required Functions

In [4]:
def adjust_brightness_contrast(image, brightness=40, contrast=1.0):
    # Convert to float to prevent clipping
    img = image.astype(np.float32)
    # Adjust brightness and contrast
    img = img * contrast + brightness
    # Clip to keep pixel values between 0 and 255 and convert back to uint8
    img = np.clip(img, 0, 255).astype(np.uint8)
    return img

In [5]:
def normalization_landmarks(landmarks):
    # Normalize landmarks relative to the wrist (landmark 0) for each frame
    normalized_landmark_data = []
    for frame in landmarks:
        # Extract wrist coordinates
        wrist_x, wrist_y, wrist_z = frame[0], frame[1], frame[2]

        # Normalize each landmark in the frame relative to the wrist
        normalized_frame = []
        for i in range(0, len(frame), 3):  # Iterate over (x, y, z) coordinates
            normalized_x = frame[i] - wrist_x
            normalized_y = frame[i + 1] - wrist_y
            normalized_z = frame[i + 2] - wrist_z
            normalized_frame.extend([normalized_x, normalized_y, normalized_z])

        normalized_landmark_data.append(normalized_frame)

    # Convert to numpy array
    normalized_landmark_data = np.array(normalized_landmark_data)
    
    return normalized_landmark_data

In [6]:
def predict_image(directory):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,min_detection_confidence=0.4)
    mp_drawing = mp.solutions.drawing_utils

    img = cv2.imread(directory)
    img = adjust_brightness_contrast(img, 40, 1)
    img_rbg =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rbg)

    sequence = []
    sequence_length = 1

    if result.multi_hand_landmarks:
        landmarks = []
        for lm in result.multi_hand_landmarks[0].landmark:
            landmarks.extend([lm.x, lm.y, lm.z])

        # Draw hand landmarks on the frame
        mp_drawing.draw_landmarks(
            img,
            result.multi_hand_landmarks[0],
            mp_hands.HAND_CONNECTIONS
        )

        # Append new frame landmarks to sequence

        sequence.append(landmarks)
        if len(sequence) > sequence_length:
            sequence.pop(0)

        if len(sequence) == sequence_length:
            sequence_input = np.array(sequence)
            sequence_input = normalization_landmarks(sequence_input)
            sequence_input = sequence_input.flatten()[np.newaxis, ..., np.newaxis]
            prediction = model.predict(sequence_input)
            predicted_label_index = np.argmax(prediction)
            predicted_label = label_encoder.inverse_transform([predicted_label_index])
            confidence = prediction[0][predicted_label_index]

    plt.axis('off')
    plt.imshow(img)
    return predicted_label

In [7]:
def evaluate_model(test_data_dir):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.4)


        labels1 = []
        landmark_data1 = []

        for letter in os.listdir(test_data_dir):

            letter_dir = os.path.join(test_data_dir, letter)
            for i, img_path in enumerate(os.listdir(letter_dir)):

                img = cv2.imread(os.path.join(letter_dir, img_path))
                img = adjust_brightness_contrast(img, 40, 1)

                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                results = hands.process(img_rgb)


                if results.multi_hand_landmarks:
                    landmarks = []
                    for lm in results.multi_hand_landmarks[0].landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])
                    landmark_data1.append(landmarks)
                    labels1.append(letter)
        landmark_data1 = np.array(landmark_data1)
        labels1 = np.array(labels1)


        # Normalize landmarks between 0 and 1
        landmark_data1 = normalization_landmarks(landmark_data1)

        # Encode labels as integers and convert to categorical

        labels_encoded1 = label_encoder.transform(labels1)
        labels_categorical1 = to_categorical(labels_encoded1)
        landmark_data1 = np.reshape(landmark_data1,(-1,63,1))

        return model.evaluate(landmark_data1,labels_categorical1)

## MediaPipe Landmark Creation for Training Data

In [ ]:
# Initialize MediaPipe Hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.2)
mp_drawing = mp.solutions.drawing_utils

# Path where images are stored for each letter in the ASL alphabet
data_dir = "../raw_data/asl_alphabet_train/asl_alphabet_train" # Change according to local dataset

landmark_data = []
labels = []

In [ ]:
# Collect landmarks for each letter
# This function does not normalize inside. We need to run Normalization function outside
for letter in os.listdir(data_dir):
    #if letter=="C":
    #    break
    letter_dir = os.path.join(data_dir, letter)
    for i, img_path in enumerate(os.listdir(letter_dir)):
        #if i >= 300:
        #    break
        img = cv2.imread(os.path.join(letter_dir, img_path))
        img = adjust_brightness_contrast(img, 40, 1)
        #img = adjust_brightness_contrast(img, 20, 0.7)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        
        # Check for hand landmarks and store them
        if results.multi_hand_landmarks:
            landmarks = []
            for lm in results.multi_hand_landmarks[0].landmark:
                landmarks.extend([lm.x, lm.y, lm.z])  # Flattened landmark vector
            landmark_data.append(landmarks)
            labels.append(letter)  # Store the label (e.g., "A", "B", etc.)
    print(letter)

In [183]:
# Save the arrays to .npy files
# landmark_data is NOT normalized until now. Only what MediaPipe does.
np.save("landmark_data_v_large.npy", landmark_data)
np.save("labels_v_large.npy", labels)

## Preprocessing

### If you already have landmark_data and labels you can start running the following:

In [8]:
#labels = np.load("labels.npy")
#landmark_data = np.load("landmark_data.npy")

In [9]:
#Label encoding as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [10]:
#Applying Normalization function:
normalized_landmarks = normalization_landmarks(landmark_data)

### Model Prep

This is only if required but as we are using a different test set I will not test/train split

In [146]:
# Split into train and test sets if needed
X_train, X_test, y_train, y_test = train_test_split(normalized_landmarks, labels_categorical, stratify=labels_categorical, test_size=0.2, random_state=42)

## Modelling

### Building & Compiling

In [188]:
#Testing Model
model = Sequential([
    # First convolutional block
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Fully connected layers
    Flatten(),
    
    # Additional dense layer before the output
    Dense(64, activation='relu'),
    Dropout(0.2),
    
    # Fully connected layers
    Dense(256, activation='relu'),
    Dropout(0.6),
    
    # Output layer with softmax activation for classification
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [193]:
#Testing Model
model = Sequential([
    # First convolutional block
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Second convolutional block
    Conv1D(128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),
    
    # Third convolutional block
    Conv1D(256, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),

    # Fully connected layers
    Flatten(),
    
    # Additional dense layer before the output
    Dense(64, activation='relu'),
    Dropout(0.2),
    
    # Fully connected layers
    Dense(256, activation='relu'),
    Dropout(0.6),
    
    # Output layer with softmax activation for classification
    Dense(len(label_encoder.classes_), activation='softmax')
])


/Users/wabe/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [153]:
#Original Model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Dropout(0.2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [196]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Fitting

In [197]:
es = EarlyStopping(patience = 2, 
                   restore_best_weights=True)

In [198]:
# Train the model
model.fit(x, 
          y, 
          epochs=5, 
          batch_size=64, 
          validation_data=(X_test[..., np.newaxis], y_test), 
          callbacks=es)

Epoch 1/5
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 48s 13ms/step - accuracy: 0.7231 - loss: 0.8944 - val_accuracy: 0.9771 - val_loss: 0.0695
Epoch 2/5
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 45s 13ms/step - accuracy: 0.9484 - loss: 0.1643 - val_accuracy: 0.9807 - val_loss: 0.0592
Epoch 3/5
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 43s 12ms/step - accuracy: 0.9593 - loss: 0.1357 - val_accuracy: 0.9823 - val_loss: 0.0490
Epoch 4/5
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 44s 12ms/step - accuracy: 0.9644 - loss: 0.1177 - val_accuracy: 0.9875 - val_loss: 0.0401
Epoch 5/5
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 45s 13ms/step - accuracy: 0.9686 - loss: 0.1071 - val_accuracy: 0.9892 - val_loss: 0.0371


### Saving

In [201]:
save_model(model, 'asl_new_model?.keras')

## Model Testing

### If you already have a model. You can load it up here:

In [ ]:
# Load the trained model
# model = tf.keras.models.load_model("old_asl_sign_language_model.keras")

In [151]:
#0.5-0.6
evaluate_model("raw_data/test_set_pics")

I0000 00:00:1730927879.223193 6326681 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1730927879.231553 6457570 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730927879.239616 6457570 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/wabe/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6003 - loss: 2.4117 


[1.9537979364395142, 0.6811594367027283]

In [200]:
evaluate_model("raw_data/test_set_pics")

I0000 00:00:1730930179.071587 6326681 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1730930179.078971 6484375 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730930179.083731 6484374 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/wabe/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5698 - loss: 1.8008


[1.5537683963775635, 0.6521739363670349]